In [1]:
!pip install kaggle --upgrade

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"sibbir","key":"c9b1f65daec00b23325e5e866738f3b6"}'}

In [3]:
import os
import zipfile

# Create a Kaggle directory
os.makedirs("/root/.kaggle", exist_ok=True)

# Move kaggle.json to the correct directory
!mv kaggle.json /root/.kaggle/

# Set correct permissions
!chmod 600 /root/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d thienkhonghoc/affectnet -p /content

Dataset URL: https://www.kaggle.com/datasets/thienkhonghoc/affectnet
License(s): unknown


In [5]:
!kaggle datasets download -d thienkhonghoc/affectnet

Dataset URL: https://www.kaggle.com/datasets/thienkhonghoc/affectnet
License(s): unknown
affectnet.zip: Skipping, found more recently modified local copy (use --force to force download)


In [6]:
!unzip -q /content/affectnet.zip -d /content/affectnet > /dev/null 2>&1

In [7]:
!pip install torch torchvision timm matplotlib tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 93.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 79.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import DataLoader, WeightedRandomSampler
import torchvision.datasets as datasets
from PIL import Image
import os
import numpy as np
import torch.nn.functional as F
from torch.cuda.amp import GradScaler, autocast
import time
import cv2
import albumentations as A
from albumentations.pytorch import ToTensorV2
import timm
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from torch.optim.swa_utils import AveragedModel, update_bn
import albumentations as A
from albumentations.pytorch import ToTensorV2

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Paths to dataset
train_data_path = "/content/affectnet/AffectNet/train"
val_data_path = "/content/affectnet/AffectNet/val"

# Optimize CUDA performance
torch.backends.cudnn.benchmark = True

# Define number of classes
num_classes = 8

# Custom dataset with face preprocessing
class FacialEmotionDataset(datasets.ImageFolder):
    def __init__(self, root, transform=None, face_detector=None):
        super(FacialEmotionDataset, self).__init__(root=root, transform=None)
        self.transform = transform
        self.face_detector = face_detector

    def __getitem__(self, index):
        path, target = self.samples[index]

        try:
            # Load image
            img = cv2.imread(path)
            if img is None:
                # Fallback if image cannot be loaded
                img = np.zeros((224, 224, 3), dtype=np.uint8)
                print(f"Warning: Could not load image {path}")
            else:
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

                # Apply face detection and alignment if available
                if self.face_detector is not None:
                    # You can implement face detection here
                    # For simplicity, we'll skip actual implementation
                    pass

                # Apply transformations
                if self.transform is not None:
                    transformed = self.transform(image=img)
                    img = transformed["image"]

            return img, target

        except Exception as e:
            print(f"Error processing {path}: {str(e)}")
            # Return a blank image as fallback
            if self.transform:
                blank = np.zeros((224, 224, 3), dtype=np.uint8)
                transformed = self.transform(image=blank)
                return transformed["image"], target
            else:
                return torch.zeros((3, 224, 224)), target

# Define augmentations with Albumentations (more efficient)
train_transform = A.Compose([
    A.Resize(256, 256),
    A.RandomResizedCrop(height=224, width=224, scale=(0.8, 1.0), size=(224, 224)), # Added size parameter
    A.HorizontalFlip(p=0.5),
    A.Affine(scale=0.15, translate_percent=0.1, rotate=15, p=0.5),
    A.OneOf([
        A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2),
        A.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=30, val_shift_limit=20),
    ], p=0.7),
    A.OneOf([
        A.MotionBlur(p=0.2),
        A.MedianBlur(blur_limit=3, p=0.1),
        A.GaussianBlur(blur_limit=3, p=0.1),
    ], p=0.3),
    A.CoarseDropout(
        max_holes=8, max_height=16, max_width=16,
        min_holes=1, min_height=8, min_width=8, p=0.3
    ),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

# Add the missing validation transform
val_transform = A.Compose([
    A.Resize(224, 224),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

# Function to filter corrupt images
def filter_corrupt_images(dataset_path):
    valid_samples = []
    class_dirs = [d for d in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, d))]

    for class_idx, class_dir in enumerate(class_dirs):
        class_path = os.path.join(dataset_path, class_dir)
        for img_name in os.listdir(class_path):
            img_path = os.path.join(class_path, img_name)
            try:
                img = Image.open(img_path).convert("RGB")
                img.verify()  # Verify image
                valid_samples.append((img_path, class_idx))
            except Exception as e:
                print(f"Corrupt image removed: {img_path}, Error: {str(e)}")

    return valid_samples

# Load datasets with custom preprocessing
print("Loading and preprocessing datasets...")
train_dataset = FacialEmotionDataset(root=train_data_path, transform=train_transform)
val_dataset = FacialEmotionDataset(root=val_data_path, transform=val_transform)

# Compute class weights and setup weighted sampling for imbalance
class_counts = np.bincount(train_dataset.targets)
# Ensure we have counts for all classes
if len(class_counts) < num_classes:
    class_counts = np.pad(class_counts, (0, num_classes - len(class_counts)), 'constant')

# Calculate class weights (inverse frequency)
class_weights = 1.0 / np.array(class_counts)
class_weights = class_weights / np.sum(class_weights) * num_classes
class_weights_tensor = torch.FloatTensor(class_weights).to(device)

print(f"Class distribution: {class_counts}")
print(f"Class weights: {class_weights}")

# Create weighted sampler for imbalanced dataset
sample_weights = [class_weights[t] for t in train_dataset.targets]
sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)

# Create data loaders
num_workers = min(8, os.cpu_count() or 4)
train_loader = DataLoader(
    train_dataset,
    batch_size=32,  # Smaller batch size for better generalization
    sampler=sampler,  # Use weighted sampler
    num_workers=num_workers,
    pin_memory=True,
    drop_last=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=64,
    shuffle=False,
    num_workers=num_workers,
    pin_memory=True
)

# Define model - Try EfficientNetV2 which works well for facial features
print("Initializing model...")
model = timm.create_model('efficientnetv2_s', pretrained=False, num_classes=num_classes)
model = model.to(device)

# Stochastic Weight Averaging
swa_model = AveragedModel(model)

# Define loss function - Focal Loss to handle class imbalance
class FocalLoss(nn.Module):
    def __init__(self, weight=None, gamma=2.0, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.weight = weight
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, input, target):
        ce_loss = F.cross_entropy(input, target, weight=self.weight, reduction='none')
        pt = torch.exp(-ce_loss)
        focal_loss = ((1 - pt) ** self.gamma) * ce_loss

        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss

# Define mixed loss (Focal + Label Smoothing)
class MixedLoss(nn.Module):
    def __init__(self, weight=None, gamma=2.0, alpha=0.8, smoothing=0.1):
        super(MixedLoss, self).__init__()
        self.focal = FocalLoss(weight=weight, gamma=gamma)
        self.smoothing = smoothing
        self.alpha = alpha  # Weight for focal loss
        self.num_classes = len(weight) if weight is not None else 8

    def forward(self, input, target):
        # Focal loss
        focal_loss = self.focal(input, target)

        # Label smoothing
        smooth_target = torch.zeros_like(input).scatter_(
            1, target.unsqueeze(1), 1.0
        )
        smooth_target = smooth_target * (1.0 - self.smoothing) + self.smoothing / self.num_classes
        smooth_loss = -(smooth_target * F.log_softmax(input, dim=1)).sum(dim=1).mean()

        # Combine losses
        return self.alpha * focal_loss + (1 - self.alpha) * smooth_loss

# Initialize the loss function
criterion = MixedLoss(weight=class_weights_tensor, gamma=2.0, alpha=0.8, smoothing=0.1)

# Use different learning rates for backbone and classifier
backbone_params = []
classifier_params = []

for name, param in model.named_parameters():
    if 'head' in name or 'classifier' in name or 'fc' in name:
        classifier_params.append(param)
    else:
        backbone_params.append(param)

# Optimizer with differential learning rates
optimizer = optim.AdamW([
    {'params': backbone_params, 'lr': 1e-5},  # Lower LR for backbone
    {'params': classifier_params, 'lr': 5e-4}  # Higher LR for new layers
], weight_decay=1e-4)

# Learning rate scheduler - One cycle policy gives good results
scheduler = optim.lr_scheduler.OneCycleLR(
    optimizer,
    max_lr=[1e-5, 5e-4],  # Different max_lr for different param groups
    steps_per_epoch=len(train_loader),
    epochs=30,
    pct_start=0.3,
    div_factor=25,
    final_div_factor=1000
)

# Gradient scaler for mixed precision training
scaler = GradScaler()

# Training parameters
num_epochs = 30
best_val_acc = 0.0
early_stopping_patience = 7
epochs_without_improvement = 0
accumulation_steps = 2  # Gradient accumulation for effective larger batch size
swa_start = 5  # Start SWA after this many epochs

# Training Loop
print("Starting training...")
start_time = time.time()

train_losses = []
train_accs = []
val_accs = []

for epoch in range(1, num_epochs + 1):
    # Training phase
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)

        # Mixed precision training
        with autocast():
            outputs = model(images)
            loss = criterion(outputs, labels)

        # Scale loss and compute gradients
        scaler.scale(loss).backward()

        # Gradient accumulation
        if (i + 1) % accumulation_steps == 0:
            # Unscale gradients for clipping
            scaler.unscale_(optimizer)

            # Clip gradients to prevent exploding gradients
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

            # Update parameters and clear gradients
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()

            # Update LR
            scheduler.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct_train += (predicted == labels).sum().item()
        total_train += labels.size(0)

        # Print progress every 50 batches
        if (i + 1) % 50 == 0:
            print(f"Epoch [{epoch}/{num_epochs}], Batch [{i+1}/{len(train_loader)}], "
                  f"Loss: {loss.item():.4f}, "
                  f"LR: {scheduler.get_last_lr()[1]:.6f}")

    # Calculate training metrics
    epoch_loss = running_loss / len(train_loader)
    train_accuracy = 100 * correct_train / total_train
    train_losses.append(epoch_loss)
    train_accs.append(train_accuracy)

    # Validation phase
    model.eval()
    correct_val = 0
    total_val = 0
    val_loss = 0.0
    all_preds = []
    all_targets = []

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct_val += (predicted == labels).sum().item()
            total_val += labels.size(0)

            # Store predictions and targets for confusion matrix
            all_preds.extend(predicted.cpu().numpy())
            all_targets.extend(labels.cpu().numpy())

    val_accuracy = 100 * correct_val / total_val
    val_accs.append(val_accuracy)

    # Update SWA model if we're past the start epoch
    if epoch >= swa_start:
        swa_model.update_parameters(model)

    # Print epoch summary
    print(f"\nEpoch [{epoch}/{num_epochs}] Summary:")
    print(f"   Train Loss: {epoch_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%")
    print(f"   Validation Loss: {val_loss/len(val_loader):.4f}, Validation Accuracy: {val_accuracy:.2f}%")

    # Save model checkpoint
    if val_accuracy > best_val_acc:
        best_val_acc = val_accuracy
        epochs_without_improvement = 0
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'val_accuracy': val_accuracy,
        }, "affectnet_best_model.pth")

        # If we've reached a very good accuracy, also save this checkpoint specifically
        if val_accuracy >= 80.0:
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'val_accuracy': val_accuracy,
            }, f"affectnet_80plus_{epoch}_{val_accuracy:.2f}.pth")

        print(f"   New best model saved with validation accuracy: {val_accuracy:.2f}%")
    else:
        epochs_without_improvement += 1
        print(f"   No improvement for {epochs_without_improvement} epochs. Best: {best_val_acc:.2f}%")

    # Early stopping check
    if epochs_without_improvement >= early_stopping_patience:
        print(f"Early stopping triggered after {epoch} epochs")
        break

    print("\n" + "-"*70 + "\n")

# Update BatchNorm statistics for SWA model
if epoch >= swa_start:
    print("Updating batch norm statistics for SWA model...")
    update_bn(train_loader, swa_model, device=device)

    # Evaluate SWA model
    swa_model.eval()
    correct_swa = 0
    total_swa = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = swa_model(images)
            _, predicted = torch.max(outputs, 1)
            correct_swa += (predicted == labels).sum().item()
            total_swa += labels.size(0)

    swa_accuracy = 100 * correct_swa / total_swa
    print(f"SWA Model Validation Accuracy: {swa_accuracy:.2f}%")

    # Save SWA model if it's better
    if swa_accuracy > best_val_acc:
        torch.save({
            'model_state_dict': swa_model.state_dict(),
            'val_accuracy': swa_accuracy,
        }, "affectnet_swa_best.pth")
        print(f"SWA model saved with validation accuracy: {swa_accuracy:.2f}%")

# Final evaluation with confusion matrix
print("\nGenerating final evaluation metrics...")
model.load_state_dict(torch.load("affectnet_best_model.pth")['model_state_dict'])
model.eval()

all_preds = []
all_targets = []

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_targets.extend(labels.cpu().numpy())

# Create confusion matrix
conf_matrix = confusion_matrix(all_targets, all_preds)
class_names = [str(i) for i in range(num_classes)]  # Or get actual class names

# Print classification report
print("\nClassification Report:")
print(classification_report(all_targets, all_preds, target_names=class_names))

# Plot training history
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(train_losses)
plt.title('Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')

plt.subplot(1, 2, 2)
plt.plot(train_accs, label='Train')
plt.plot(val_accs, label='Validation')
plt.title('Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.legend()
plt.tight_layout()
plt.savefig('training_history.png')

print(f"\nTraining completed in {(time.time() - start_time) / 60:.2f} minutes")
print(f"Best validation accuracy: {best_val_acc:.2f}%")

Using device: cuda
Loading and preprocessing datasets...
Class distribution: [5000 3803 5000 5000 5000 5000 5000 3750]
Class weights: [0.9250603  1.21622443 0.9250603  0.9250603  0.9250603  0.9250603
 0.9250603  1.23341374]
Initializing model...


<ipython-input-18-207ef0692104>:85: UserWarning: Argument(s) 'height, width' are not valid for transform RandomResizedCrop
  A.RandomResizedCrop(height=224, width=224, scale=(0.8, 1.0), size=(224, 224)), # Added size parameter
<ipython-input-18-207ef0692104>:97: UserWarning: Argument(s) 'max_holes, max_height, max_width, min_holes, min_height, min_width' are not valid for transform CoarseDropout
  A.CoarseDropout(


Starting training...


<ipython-input-18-207ef0692104>:254: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
<ipython-input-18-207ef0692104>:283: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:227: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(


Epoch [1/30], Batch [50/1173], Loss: 1.5806, LR: 0.000020
Epoch [1/30], Batch [100/1173], Loss: 2.2326, LR: 0.000020
Epoch [1/30], Batch [150/1173], Loss: 2.6892, LR: 0.000020
Epoch [1/30], Batch [200/1173], Loss: 2.0897, LR: 0.000020
Epoch [1/30], Batch [250/1173], Loss: 1.8558, LR: 0.000020
Epoch [1/30], Batch [300/1173], Loss: 2.0571, LR: 0.000020
Epoch [1/30], Batch [350/1173], Loss: 2.0974, LR: 0.000020
Epoch [1/30], Batch [400/1173], Loss: 1.8550, LR: 0.000020
Epoch [1/30], Batch [450/1173], Loss: 2.1184, LR: 0.000021
Epoch [1/30], Batch [500/1173], Loss: 2.0097, LR: 0.000021
Epoch [1/30], Batch [550/1173], Loss: 1.9710, LR: 0.000021
Epoch [1/30], Batch [600/1173], Loss: 1.9536, LR: 0.000021
Epoch [1/30], Batch [650/1173], Loss: 1.9366, LR: 0.000021
Epoch [1/30], Batch [700/1173], Loss: 1.7858, LR: 0.000021
Epoch [1/30], Batch [750/1173], Loss: 1.6621, LR: 0.000021
Epoch [1/30], Batch [800/1173], Loss: 1.8202, LR: 0.000022
Epoch [1/30], Batch [850/1173], Loss: 1.9346, LR: 0.00002